<div align="center">

  <a href="https://ultralytics.com/yolo" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>

  [中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [Türkçe](https://docs.ultralytics.com/tr/) | [Tiếng Việt](https://docs.ultralytics.com/vi/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml"><img src="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml/badge.svg" alt="Ultralytics CI"></a>
  <a href="https://colab.research.google.com/github/ultralytics/notebooks/blob/main/notebooks/how-to-count-the-objects-using-ultralytics-yolo.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  
  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>
  
  This notebook demonstrates how to use Florence 2 with Ultralytics <a href="https://github.com/ultralytics/ultralytics">YOLO</a> annotators for object detection, image segmentation, and generating visualizations from text prompts, such as image captioning. We aim to provide resources that help you maximize the potential of Florence-2. If you need assistance, feel free to raise an issue on <a href="https://github.com/ultralytics/ultralytics">GitHub</a> or join our <a href="https://ultralytics.com/discord">Discord</a> community for discussions and support!

# What is Florence-2?

Microsoft released the Florence-2 model last year, It is an advanced vision foundation model that uses a prompt-based approach to handle a wide range of vision and vision-language tasks. It can interpret simple text prompts to perform tasks like captioning, object detection, and segmentation.
  
It leverages the FLD-5B dataset, containing 5.4 billion annotations across 126 million images, to master multi-task learning. The model's sequence-to-sequence architecture enables it to excel in both zero-shot and fine-tuned settings, proving to be a competitive vision foundation model.
<img src="https://github.com/user-attachments/assets/28e24618-9777-4dda-af81-8be730993976" alt="Florence 2 Arcitecture" />

## Setup

To get started, we need to install the `ultralytics` and `transformers` libraries. 🚀

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [53]:
# !pip install transformers ultralytics

from ultralytics.utils.downloads import safe_download
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.solutions.solutions import SolutionAnnotator

from transformers import AutoProcessor, AutoModelForCausalLM

from PIL import Image
import numpy as np
import cv2
import torch

## Download the `Florence-2-large` model using the `transformers` library.

In [ ]:
model_id = "microsoft/Florence-2-large"

# Ensure the runtime is set to GPU in Colab.
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True,
                                             torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

## Inference using Florence-2

In [74]:
def inference(image, task_prompt, text_input=None):
    """
    Performs inference using the given image and task prompt.

    Args:
        image (PIL.Image or tensor): The input image for processing.
        task_prompt (str): The prompt specifying the task for the model.
        text_input (str, optional): Additional text input to refine the task prompt.

    Returns:
        dict: The model's processed response after inference.
    """

    # Combine task prompt with additional text input if provided
    prompt = task_prompt if text_input is None else task_prompt + text_input

    # Generate the input data for model processing from the given prompt and image
    inputs = processor(
        text=prompt,  # Text input for the model
        images=image,  # Image input for the model
        return_tensors="pt"  # Return PyTorch tensors
    ).to('cuda', torch.float16)  # Move inputs to GPU with float16 precision

    # Generate model predictions (token IDs)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"].cuda(),  # Convert text input IDs to CUDA
        pixel_values=inputs["pixel_values"].cuda(),  # Convert image pixel values to CUDA
        max_new_tokens=1024,  # Set maximum number of tokens to generate
        early_stopping=False,  # Disable early stopping
        do_sample=False,  # Use deterministic inference
        num_beams=3,  # Set beam search width for better predictions
    )

    # Decode generated token IDs into text
    generated_text = processor.batch_decode(
        generated_ids,  # Generated token IDs
        skip_special_tokens=False  # Retain special tokens in output
    )[0]  # Extract first result from batch

    # Post-process the generated text into a structured response
    parsed_answer = processor.post_process_generation(
        generated_text,  # Raw generated text
        task=task_prompt,  # Task type for post-processing
        image_size=(image.width, image.height)  # Original image dimensions for scaling output
    )

    return parsed_answer  # Return the final processed output

## Download and read the Image  

You can use any image of your choice. In this example, we'll download `bus.jpg` from [Ultralytics](https://ultralytics.com/) assets and use it for further processing.

In [6]:
def read_image():
  image_name = "bus.jpg"  # or "zidane.jpg"

  # Download the image
  safe_download(f"https://github.com/ultralytics/assets/releases/download/v0.0.0/{image_name}")

  # Read the image using OpenCV and convert it into the PIL format
  return Image.fromarray(cv2.cvtColor(cv2.imread(f"/content/{image_name}"), cv2.COLOR_BGR2RGB))

<img src="https://github.com/user-attachments/assets/e3f77be3-4174-4310-bd33-d2955cda7a94">

## Execute pre-defined tasks without extra inputs

With the model loaded into memory, the input image ready for processing, and functions in place for preprocessing and post-processing, it's time to play some magic with Florence-2. 🚀

### 1- Object detection  

Florence-2 supports object detection, allowing us to efficiently identify and recognize various objects within an image.

```json
# Result format 😀
{
    "<OD>": {
        "bboxes": [[x1, y1, x2, y2], ...],
        "labels": ["label1", "label2", ...]
    }
}
```

In [ ]:
task_prompt = '<OD>'
image = read_image()

results = inference(image, task_prompt)['<OD>']

# Plot the results on an image
annotator = Annotator(image)  # initialize Ultralytics annotator

for idx, (box, label) in enumerate(zip(results['bboxes'], results['labels'])):
  annotator.box_label(box, label=label, color=colors(idx, True))

Image.fromarray(annotator.result())  # display the output

![Object detection using florence-2](https://github.com/user-attachments/assets/6bf6820b-8eb5-44f2-a626-64d76c92b981)

### 2- Caption  

- Automatically generate captions for the input image without needing a custom prompt. By default, it provides three levels of captions: `CAPTION`, `DETAILED_CAPTION`, and `MORE_DETAILED_CAPTION`.  

- It has been observed that while `DETAILED_CAPTION` and `MORE_DETAILED_CAPTION` may not always be entirely accurate, `CAPTION` tends to be more precise.

In [9]:
task_prompt = '<CAPTION>'
inference(image, task_prompt)

{'<CAPTION>': 'A group of people walking down a street next to a bus.'}

In [10]:
task_prompt = '<DETAILED_CAPTION>'
inference(image, task_prompt)

{'<DETAILED_CAPTION>': 'The image shows a group of people walking down a street next to a bus, with a building in the background featuring windows, railings, doors, and balconies, as well as trees and a sign board. The people are wearing different types of clothing, suggesting that the image is related to the use of augmented reality in Madrid.'}

In [11]:
task_prompt = '<MORE_DETAILED_CAPTION>'
inference(image, task_prompt)

{'<MORE_DETAILED_CAPTION>': 'The image shows a blue and white bus on a street with a group of people walking on the sidewalk in front of it. The bus has the words "man" and "trousers" written on it, along with a logo of a leaf and the text "cero emisiones" which translates to "man". There are also several labels on the bus, including "coat", "trouser", "footwear", "wheel", and "focwear". The people in the image appear to be of different ages and genders, and they are dressed in casual clothing. The street is lined with buildings and there is a tree on the left side of the image.'}

### 3- Dense region caption

For more precise results, such as specific locations, cities, places, or object types, we can use dense region captioning. Instead of a general caption like `"a bus in an image,"` we can obtain a more detailed description, such as `"an electric bus in Madrid, Spain."`

```json
# Dense region caption results format 😀
{
    "<DENSE_REGION_CAPTION>": {
        "bboxes": [[x1, y1, x2, y2], ...],
        "labels": ["label1", "label2", ...]
    }
}
```


In [ ]:
task_prompt = '<DENSE_REGION_CAPTION>'
image = read_image()

results = inference(image, task_prompt)['<DENSE_REGION_CAPTION>']

# Plot the results on an image
annotator = Annotator(image)  # initialize Ultralytics annotator

for idx, (box, label) in enumerate(zip(results['bboxes'], results['labels'])):
  annotator.box_label(box, label=label, color=colors(idx, True))

Image.fromarray(annotator.result())  # display the output

![Dense region captioning using florence-2](https://github.com/user-attachments/assets/b10efb49-f8a8-46f6-aa62-36782d824d9f)

### 4- Region proposal

Region proposal refers to the process of identifying areas in an image that are likely to contain objects. It helps in object detection by suggesting potential object locations before classification and bounding box refinement.

```json
# Region proposal results format 🎉
{
    "<REGION_PROPOSAL>": {
        "bboxes": [[x1, y1, x2, y2], ...],
        "labels": ["label1", "label2", ...]
    }
}
```

In [ ]:
task_prompt = '<REGION_PROPOSAL>'
image = read_image()

results = inference(image, task_prompt)['<REGION_PROPOSAL>']

# Plot the results on an image
annotator = Annotator(image)  # initialize Ultralytics annotator
for idx, box in enumerate(results['bboxes']):
  x1, y1, x2, y2 = box
  if x1 > x2:
        x1, x2 = x2, x1  # Swap x-coordinates if needed
  if y1 > y2:
        y1, y2 = y2, y1  # Swap y-coordinates if needed

  # Update the bounding box correctly
  annotator.box_label([x1, y1, x2, y2], color=colors(idx, True))

Image.fromarray(annotator.result())  # display the output

![Region proposal using florence-2](https://github.com/user-attachments/assets/b0e3e004-f9f6-4937-9ea9-431be56e1476)

## Execute pre-defined tasks that require additional inputs

Here, we can provide a custom prompt to generate precise results tailored to specific requirements.

### Phrase grounding

We can also extract specific results i.e. `a bus` or `Madrid bus` by passing additional text_input.

```json
# Phrase grounding results format: 🚀
{
    "<CAPTION_TO_PHRASE_GROUNDING>": {
        "bboxes": [[x1, y1, x2, y2], ...],
        "labels": ["label1", "label2", ...]
    }
}
```

In [ ]:
task_prompt = '<CAPTION_TO_PHRASE_GROUNDING>'
image = read_image()

results = inference(image, task_prompt, text_input="a madrid bus, glasses")['<CAPTION_TO_PHRASE_GROUNDING>']

# Plot the results on an image
annotator = Annotator(image)  # initialize Ultralytics annotator

for idx, (box, label) in enumerate(zip(results['bboxes'], results['labels'])):
  annotator.box_label(box, label=label, color=colors(idx, True))

Image.fromarray(annotator.result())  # display the output

![Captions to phrase grouding using florence-2](https://github.com/user-attachments/assets/4c2dee5b-afd2-4d99-bc8d-bf24b93937fb)

### Referring to expression segmentation

We can use Florence-2 for segmentation based on specific expression.

```json
# Output format 😃

# one object is represented by a list of polygons. each polygon is [x1, y1, x2, y2, ..., xn, yn]
{
    "<REFERRING_EXPRESSION_SEGMENTATION>": {
        {'Polygons': [[[polygon]], ...],
        "labels": ["label1", "label2", ...]
    }
}
```

In [ ]:
task_prompt = '<REFERRING_EXPRESSION_SEGMENTATION>'
image = read_image()

results = inference(image, task_prompt, text_input="person with black coat showoff")['<REFERRING_EXPRESSION_SEGMENTATION>']

# segmentation mask function required the numpy array image, not PIL based image.
annotator = SolutionAnnotator(np.array(image, dtype=np.uint8))

for idx, (polygons, label) in enumerate(zip(results['polygons'], results['labels'])):

    for _polygon in polygons:
        _polygon = np.array(_polygon).reshape(-1, 2)

        if len(_polygon) < 3:
          print('Invalid polygon:', _polygon)
          continue

        annotator.segmentation_mask(_polygon, label=label, mask_color=colors(idx, True))

Image.fromarray(annotator.result())  # display the output

![Captions to phrase grouding using florence-2](https://github.com/user-attachments/assets/c691c040-4dbb-402a-9da2-77f89d9af18a)

### Region to segmentation

Here, we can segment a specific area by providing its coordinates.

```json
<loc_x1>\<loc_y1>\<loc_x2>\<loc_y2>, [x1, y1, x2, y2] is the quantized coordinates in [0, 999].
 ```

In [ ]:
task_prompt = '<REGION_TO_SEGMENTATION>'
image = read_image()

results = inference(image, task_prompt,
                    text_input="[3.8328723907470703, 229.35601806640625,796.2098999023438, 728.4313354492188]")['<REGION_TO_SEGMENTATION>']

# segmentation mask function required the numpy array image, not PIL based image.
annotator = SolutionAnnotator(np.array(image, dtype=np.uint8))

for idx, (polygons, label) in enumerate(zip(results['polygons'], results['labels'])):

    for _polygon in polygons:
        _polygon = np.array(_polygon).reshape(-1, 2)

        if len(_polygon) < 3:
          print('Invalid polygon:', _polygon)
          continue

        annotator.segmentation_mask(_polygon, label=label, mask_color=colors(idx, True))

Image.fromarray(annotator.result())  # display the output

![Region to segmentation using florence-2](https://github.com/user-attachments/assets/0dc7aff0-a688-4a08-a895-ba05678a2dec)

### Open vocabulary detection

It enables models to detect both objects and text (OCR) without being limited to predefined categories, making it highly adaptable for real-world applications like retail, navigation, and document analysis.

```json
# Output format 😀

{ '<OPEN_VOCABULARY_DETECTION>':

      {'bboxes': [[x1, y1, x2, y2], [x1, y1, x2, y2], ...]],
      'bboxes_labels': ['label_1', 'label_2', ..],
      'polygons': [[[x1, y1, x2, y2, ..., xn, yn], [x1, y1, ..., xn, yn]], ...],
      'polygons_labels': ['label_1', 'label_2', ...]
}}
```

In [ ]:
task_prompt = '<OPEN_VOCABULARY_DETECTION>'
image = read_image()

results = inference(image, task_prompt, text_input="trees")['<OPEN_VOCABULARY_DETECTION>']

# segmentation mask function required the numpy array image, not PIL based image.
annotator = Annotator(image)

for idx, (box, label) in enumerate(zip(results['bboxes'], results['bboxes_labels'])):
    annotator.box_label(box, color=colors(idx, True), label=label)

Image.fromarray(annotator.result())  # display the output

![Open vocabulary detection using florence-2](https://github.com/user-attachments/assets/6368ac07-6083-4762-9020-b516be328d2d)

## OCR related tasks

Florence 2 can also be utilized for OCR-related tasks.

In [ ]:
# Download the image
safe_download(f"http://ecx.images-amazon.com/images/I/51UUzBDAMsL.jpg?download=true")

# Read the image using OpenCV and convert it into the PIL format
image = Image.fromarray(cv2.imread('/content/51UUzBDAMsL.jpg'))

task_prompt = '<OCR_WITH_REGION>'
results = inference(image, task_prompt)['<OCR_WITH_REGION>']

# segmentation mask function required the numpy array image, not PIL based image.
annotator = Annotator(image, line_width=2)

for idx, (box, label) in enumerate(zip(results['quad_boxes'], results['labels'])):
  box = box[:4]
  x1, y1, x2, y2 = box
  if x1 > x2:
        x1, x2 = x2, x1  # Swap x-coordinates if needed
  if y1 > y2:
        y1, y2 = y2, y1  # Swap y-coordinates if needed

  # Update the bounding box correctly
  annotator.box_label([x1, y1, x2, y2], label=label, color=colors(idx, True))

Image.fromarray(annotator.result())  # display the output

![Open vocabulary detection using florence-2](https://github.com/user-attachments/assets/1646655e-a4fd-4055-a675-56a07ddbe0ff)

## Additional Resources  

✅ Florence-2 research paper: [here](https://arxiv.org/abs/2311.06242)  
✅ Ultralytics Annotator: [here](https://docs.ultralytics.com/reference/utils/plotting/)

🌟 Explore the [Ultralytics Notebooks](https://github.com/ultralytics/notebooks/) and give them a star to boost your AI journey! 🚀

Built with 💙 by [Ultralytics](https://ultralytics.com/)  